# Phishing Detection Using BERT

## Authors
- Eli Fajardo (NVIDIA)
- Gorkem Batmaz (NVIDIA)
- Bartley Richardson, PhD (NVIDIA)

## Table of Contents 
* Introduction
* List of datasets used
* Reading in the datasets
* Initialize CLX Phishing Detection and BERT model
* Training - CLAIR FRAUDULENT EMAILS dataset
* Evaluation of CLAIR Test Set
* Training with the the SPAM_ASSASSIN dataset
* Evaluation of the SPAM_ASSASSIN Test Set
* Training with all three datasets CLAIR+SPAM_ASSASSIN+ENRON
* Evaluation of the Test Set of CLAIR+SPAM_ASSASSIN+ENRON Datasets
* References

## Introduction
Phishing is a method used by fraudsters/hackers to obtain sensitive information from email users by pretending to be from legitimate institutions/people.
Various machine learning methods are in use to detect and filter phishing/spam emails. 
In this notebook, we show how to train a *BERT language model and analyse the performance on various datasets. We have fine-tuned a pre-trained BERT model with a classification layer using HuggingFace library. 
*BERT stands for Bidirectional Encoder Representations from Transformers. The paper can be found [here.](https://arxiv.org/pdf/1810.04805.pdf)

## Datasets used
* [CLAIR-Fraudulent E-mail Corpus](https://www.kaggle.com/rtatman/fraudulent-email-corpus)
* [SPAM_ASSASSIN Dataset](https://spamassassin.apache.org/old/publiccorpus/)
* [Enron Emails](https://www.cs.cmu.edu/~enron/)

### Required Libraries

In [1]:
import cudf;
from cuml.preprocessing.model_selection import train_test_split;
from clx.analytics.sequence_classifier import SequenceClassifier;
import s3fs;
from os import path;

/opt/conda/envs/rapids/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/rapids/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/rapids/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/rapids/lib/python3.7/site-packages/tensorflow/python/framework/dty

## Reading the files

In [2]:
CLAIR_TSV = "Phishing_Dataset_Clair_Collection.tsv"
SPAM_TSV = "spam_assassin_spam_200_20021010.tsv"
EASY_HAM_TSV = "spam_assassin_easyham_200_20021010.tsv"
HARD_HAM_TSV = "spam_assassin_hardham_200_20021010.tsv"
ENRON_TSV = "enron_10000.tsv"

S3_BASE_PATH = "rapidsai-data/cyber/clx"

In [3]:
# Clair dataset
if not path.exists(CLAIR_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + CLAIR_TSV, CLAIR_TSV)
    
dfclair = cudf.read_csv(CLAIR_TSV, delimiter='\t', header=None, names=['label', 'email']).dropna()

In [4]:
# Phishing emails of the SPAM ASSASSIN dataset
if not path.exists(SPAM_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + SPAM_TSV, SPAM_TSV)
 
dfspam = cudf.read_csv(SPAM_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [5]:
# Benign emails of the SPAM ASSASSIN dataset
if not path.exists(EASY_HAM_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + EASY_HAM_TSV, EASY_HAM_TSV)
    
dfeasyham = cudf.read_csv(EASY_HAM_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [6]:
# Benign emails of the SPAM ASSASSIN dataset that are easy to be confused with phishing emails
if not path.exists(HARD_HAM_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + HARD_HAM_TSV, HARD_HAM_TSV)

dfhardham = cudf.read_csv(HARD_HAM_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [7]:
# Benign Enron emails
if not path.exists(ENRON_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + ENRON_TSV, ENRON_TSV)

dfenron = cudf.read_csv(ENRON_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [8]:
# The files contain the first 200 words of each email. The model uses only the first 128 words.

## Initialize/Load BERT model

In [9]:
seq_classifier = SequenceClassifier()
seq_classifier.init_model("bert-base-uncased")

# init_model can also load pre-trained model by passing it model directory path

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Training - CLAIR FRAUDULENT EMAILS DATASET

Clair fraudulent e-mails contain criminally deceptive information, usually with the intent of convincing the recipient to give the sender a large amount of money.*
The original dataset can be found [here](https://www.kaggle.com/rtatman/fraudulent-email-corpus). 

Split the dataset into training and test sets

In [10]:
X_train, X_test, y_train, y_test = train_test_split(dfclair, 'label', train_size=0.8)

In [11]:
seq_classifier.train_model(X_train["email"], y_train, epochs=1)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Epoch: 100%|██████████| 1/1 [00:47<00:00, 47.48s/it]

Train loss: 0.20608550182711255


Validation Accuracy: 0.9947916666666666


## Evaluation of CLAIR Test Set

In [12]:
seq_classifier.evaluate_model(X_test["email"], y_test)

0.9953897736797989

This indicates over 99% of the emails in the test dataset were correctly marked as fraudulent or benign.

## Training with SPAM_ASSASSIN dataset

SPAM_ASSASIN public mail corpus consists of three classes. The first category is "easy ham" which has benign emails; the second category is "hard ham" which has emails which are benign but are closer to spam, and finally, the spam category that includes the fraudulent emails. More info [here](https://spamassassin.apache.org/old/publiccorpus/readme.html)

Merging the spam assasin dataset

In [13]:
df_assassin = cudf.concat([dfhardham,dfeasyham,dfspam], ignore_index=True)

Split the dataset into train and test

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df_assassin, 'label', train_size=0.8)

In [15]:
seq_classifier.train_model(X_train["email"], y_train, epochs=1)

Epoch: 100%|██████████| 1/1 [00:12<00:00, 12.62s/it]

Train loss: 0.7128496293091413


Validation Accuracy: 0.9209558823529411


## Evaluation of the SPAM_ASSASSIN Test Set

In [16]:
seq_classifier.evaluate_model(X_test["email"], y_test)

0.9349470499243571

~84% of the spam assassin emails were predicted correctly as spam or benign



## Training with CLAIR+SPAM_ASSASSIN datasets

Merge the two datasets and split as train and test sets

In [17]:
df_total = cudf.concat([dfhardham,dfeasyham,dfspam,dfclair],ignore_index=True)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df_total, 'label', train_size=0.8)

In [19]:
seq_classifier.train_model(X_train["email"], y_train, epochs=1)

Epoch: 100%|██████████| 1/1 [00:57<00:00, 57.07s/it]

Train loss: 0.11770776656005898


Validation Accuracy: 0.9955357142857143


## Evaluation of the Test Set of CLAIR+SPAM_ASSASSIN Datasets

In [20]:
seq_classifier.evaluate_model(X_test["email"], y_test)

0.9947472094550229

When CLAIR and SPAM_ASSASSIN datasets are combined, the test set accuracy is over 99%, which means 99% of the emails in the test set are marked correctly as benign or spam.

## Training with all three datasets (CLAIR+SPAM_ASSASSIN+ENRON)

Enron Email Dataset was collected and prepared by the CALO Project (A Cognitive Assistant that Learns and Organizes). We use them as benign email class. 
More info on the ENRON Email Dataset can be found [here](https://www.cs.cmu.edu/~./enron/)

Merge all the datasets, split into training and test set and then tokenize the emails

In [21]:
df_total = cudf.concat([dfhardham,dfeasyham,dfspam,dfclair,dfenron],ignore_index=True)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df_total, 'label', train_size=0.8)

In [23]:
seq_classifier.train_model(X_train["email"], y_train, epochs=1)

Epoch: 100%|██████████| 1/1 [01:35<00:00, 95.50s/it]

Train loss: 0.026596252602440914


Validation Accuracy: 0.9982775590551181


## Evaluation of the Test Set of CLAIR+SPAM_ASSASSIN+ENRON Datasets

In [24]:
seq_classifier.evaluate_model(X_test["email"], y_test)

0.9980182322631788

## Conclusion

When all three datasets are combined, over 99% of the emails are labelled correctly. This shows that using a BERT-based phishing detector performs well in identifying the spam emails across these datasets. This notebook is prepared as a POC. Users can experiment with other datasets, increase the coverage and change the number of epochs to fine-tune the results on their datasets. It is also an example of how CLX can be used with huggingface and other libraries to create custom solutions

# References
* Radev, D. (2008), CLAIR collection of fraud email, ACL Data and Code Repository, ADCR2008T001, http://aclweb.org/aclwiki
* https://www.kaggle.com/rtatman/fraudulent-email-corpus *
* https://www.cs.cmu.edu/~./enron/
* https://spamassassin.apache.org/old/publiccorpus/readme.html
* https://github.com/huggingface/transformers/tree/master/examples#
* https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/
* https://github.com/ThilinaRajapakse/pytorch-transformers-classification
* https://mccormickml.com/2019/07/22/BERT-fine-tuning/